*ToDo*

02) pred_window und obs_window ausprobieren.
03) output nochmal genau anschauen.
04) was ist maskiert anschauen - und wie genau?
05) VRAM wenig ausgelastet. Batch size mal mit 320 ausprobieren, aber auch durchziehen.
06) Task pred: scheduler einfügen.
07) Analyse each var (sparsity)
08) loss per var/quality of varis loss (is this already ablation to forecast only one var)
09) Include sepsis definition
10) 

In [1]:
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random 
import joblib
from tqdm import tqdm
tqdm.pandas()
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import roc_auc_score, precision_recall_curve, auc

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchinfo import summary
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [2]:
def set_seed(seed: int) -> None:
    """
    Set the random seed for modules torch, numpy and random.

    :param seed: random seed
    """
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    if torch.cuda.is_available() and torch.cuda.device_count() > 0:
        torch.backends.cudnn.deterministic = True
        torch.cuda.manual_seed_all(seed)
set_seed(42)

## Load forecast dataset into matrices.

In [3]:

mean_std_dict = pickle.load(open("/home/mitarb/hagmann/projects/llm_circ/imputed_data/eicu-full/mean_std_dict.pickle", "rb"))
var_to_ind = pickle.load(open("/home/mitarb/hagmann/projects/llm_circ/imputed_data/eicu-full/var2ind.pickle", "rb"))
#test_input = joblib.load(open("/home/mitarb/hagmann/projects/llm_circ/imputed_data/eicu-full/imputed_test.pickle", 'rb'))
test_input = pickle.load(open("/home/mitarb/hagmann/projects/structures_compositionality/gold_data/eicu-full/gold_test.pickle", 'rb'))





In [4]:
def get_sofa(matrix, var_to_ind): #24xV matrix
    # GCS: min_eye, min_motor, min_verbal = 5, 5, 5
    #print(matrix.size())
    #raise Exception
    key ="GCS eye"
    var_to_ind = {x:i-1 for x,i in var_to_ind.items()}
    a=matrix[:, var_to_ind[key]][torch.nonzero(matrix[:, var_to_ind[key]], as_tuple=True)]
    #print(a)
    min_eye = min(matrix[:, var_to_ind[key]][torch.nonzero(matrix[:, var_to_ind[key]], as_tuple=True)]*mean_std_dict[key][1]+mean_std_dict[key][0], default=4)
    key = "GCS motor"
    min_motor = min(matrix[:, var_to_ind[key]][torch.nonzero(matrix[:, var_to_ind[key]], as_tuple=True)]*mean_std_dict[key][1]+mean_std_dict[key][0], default=6)
    key = "GCS verbal"
    min_verbal = min(matrix[:, var_to_ind[key]][torch.nonzero(matrix[:, var_to_ind[key]], as_tuple=True)]*mean_std_dict[key][1]+mean_std_dict[key][0], default=5)
    

    GCS = min_eye + min_motor + min_verbal
    if GCS > 14: GCS_sofa = 0
    elif GCS > 12: GCS_sofa = 1
    elif GCS > 9:  GCS_sofa = 2
    elif GCS > 5:  GCS_sofa = 3
    else: GCS_sofa = 4
    #print('GCS_sofa is', GCS_sofa, ';     GCS is', GCS,'; GCS eye', min_eye, '; GCS motor', min_motor, '; GCS verbal', min_verbal)

    key = "Bilirubin (Total)"
    bilir = max(matrix[:, var_to_ind[key]][torch.nonzero(matrix[:, var_to_ind[key]], as_tuple=True)]*mean_std_dict[key][1]+mean_std_dict[key][0], default=1)
    if bilir > 12: bilir_sofa = 4
    elif bilir > 6: bilir_sofa = 3
    elif bilir > 2: bilir_sofa = 2
    elif bilir > 1.2: bilir_sofa = 1
    else: bilir_sofa = 0
    #print('bilir_Sofa is', bilir_sofa, ';   bilirubin is', bilir)
    
    # Coagulation (Platelets)
    key = "Platelets"
    plate = max(matrix[:, var_to_ind[key]][torch.nonzero(matrix[:, var_to_ind[key]], as_tuple=True)]*mean_std_dict[key][1]+mean_std_dict[key][0], default=160)
    if plate > 150: plate_sofa = 0
    elif plate > 100: plate_sofa = 1
    elif plate > 50: plate_sofa = 2
    elif plate > 20: plate_sofa = 3
    else: plate_sofa = 4
    #print('plate_sofa is', plate_sofa, ';   platelet count is', plate)
    
    # print('Urinmenge 24h', sum(data_var[data_var['variable']=='Urine']['value2']))

    key = "Urine"
    urine = sum(matrix[:, var_to_ind[key]][torch.nonzero(matrix[:, var_to_ind[key]], as_tuple=True)]*mean_std_dict[key][1]+mean_std_dict[key][0])
    key = "Creatinine (Blood)"
    creat = max(matrix[:, var_to_ind[key]][torch.nonzero(matrix[:, var_to_ind[key]], as_tuple=True)]*mean_std_dict[key][1]+mean_std_dict[key][0], default=1)
    
    if (urine < 200) or (creat > 5): renal_sofa = 4
    elif  (urine < 500) or (creat > 3.5): renal_sofa = 3
    elif creat > 2.0: renal_sofa = 2
    elif creat > 1.2: renal_sofa = 1
    else: renal_sofa = 0
    #print('renal_sofa:',renal_sofa,';       urine 24:',urine,'; creat:', creat)
    
    CS_data = get_CS(matrix, var_to_ind)
    cs_sofa = CS_SOFA(CS_data)
    
    #cs_sofa = 0
    key="FiO2"
    fio2 = (matrix[:, var_to_ind[key]]*mean_std_dict[key][1]+mean_std_dict[key][0])
    key="PaO2"
    po2 = (matrix[:, var_to_ind[key]]*mean_std_dict[key][1]+mean_std_dict[key][0])
    PaO2FiO2 = 100*po2/fio2
    #print("size", PaO2FiO2.size())
    PaO2FiO2 = PaO2FiO2[torch.nonzero(PaO2FiO2, as_tuple=True)]
    pao2fio2 = min(PaO2FiO2)
    if pao2fio2<100: resp=4
    elif pao2fio2<200: resp=3
    elif pao2fio2<300:resp=2
    elif pao2fio2<400:resp=1
    else: resp=0
    return GCS_sofa, cs_sofa, resp, plate_sofa, bilir_sofa, renal_sofa

def get_CS(matrix, var_to_ind):
    #data_var = data_pat[data_pat['variable'].isin(['Dobutamine','Dopamine','Epinephrine','Norepinephrine','Weight'])]
    #data_var['value2'] = data_var['value']*data_var['std']+data_var['mean']
    
    #weight = min(data_var[data_var['variable']=='Weight']['value2'], default=80)  # set default weight to 80kg.
    key = "Weight"
    weight = 80# min(matrix[:, var_to_ind[key]][torch.nonzero(matrix[:, var_to_ind[key]], as_tuple=True)], default=80)#*mean_std_dict[key][1]+mean_std_dict[key][0], default=1)
    key = "d Dopamine ratio"
    try:
        data_dop = max(matrix[:, var_to_ind[key]][torch.nonzero(matrix[:, var_to_ind[key]], as_tuple=True)]*mean_std_dict[key][1]+mean_std_dict[key][0], default=1)
        data_dop = data_dop /60/weight*1000
    except:
        data_dop = 0

    key = "d Dobutamine ratio"
    
    try:
        data_dobu = max(matrix[:, var_to_ind[key]][torch.nonzero(matrix[:, var_to_ind[key]], as_tuple=True)]*mean_std_dict[key][1]+mean_std_dict[key][0], default=1)
        data_dobu = data_dobu  /60/weight*1000
    except:
        data_dobu = 0
    key = "d Epinephrine ratio"
    
    try:
        data_epi = max(matrix[:, var_to_ind[key]][torch.nonzero(matrix[:, var_to_ind[key]], as_tuple=True)]*mean_std_dict[key][1]+mean_std_dict[key][0], default=1)
        data_epi = data_epi  /60/weight*1000
    except:
        data_epi = 0
    key = "d Norepinephrine ratio"
    
    try:
        data_nore = max(matrix[:, var_to_ind[key]][torch.nonzero(matrix[:, var_to_ind[key]], as_tuple=True)]*mean_std_dict[key][1]+mean_std_dict[key][0], default=1)
        data_nore = data_nore /60/weight*1000
    except:
        data_nore = 0
        


    key = "SBP"
    SBP = (matrix[:, var_to_ind[key]]*mean_std_dict[key][1]+mean_std_dict[key][0])

    key = "DBP"
    DBP = (matrix[:, var_to_ind[key]]*mean_std_dict[key][1]+mean_std_dict[key][0])

    MAP = 2/3 * DBP + 1/3 * SBP
    MAP = min(MAP[torch.nonzero(MAP, as_tuple=True)], default=100)
                 
        
    return MAP, data_dop, data_dobu, data_epi, data_nore 
    
def CS_SOFA(data):
    map = data[0]
    dop, dobu, epi, nore = data[1:5]
    # print('CS data: mdden', data)
    if (dop > 15) or (epi > 0.1) or (nore > 0.01): CS = 4
    elif (dop > 5) or (epi > 0) or (nore > 0): CS = 3
    elif (dop > 0) or (dobu > 0): CS = 2
    elif map < 70: CS = 1
    else: CS = 0
    # print('CS Sofa is:', CS)
    return CS 
    
factor_keys =["GCS eye", "GCS motor", "GCS verbal", "Bilirubin (Total)", "Platelets", "Urine", "Creatinine (Blood)", "FiO2", "PaO2", "d Dopamine ratio", "d Dobutamine ratio",
"d Epinephrine ratio", "d Norepinephrine ratio", "SBP", "DBP"]
factor_indices = sorted([var_to_ind[x]-1 for x in factor_keys])

# Crossinteractions

# 8 Sofa Analysis IMS Informer TF

In [5]:
d_model, e_layers, d_layers, d_ff = 256, 2, 2, 2048
number_of_epochs = 100
V, D = 98, 17
fore_max_len = 2640
print('loading of joblib completed')



import argparse
parser = argparse.ArgumentParser(description='Autoformer & Transformer family for Time Series Forecasting')

# basic config
parser.add_argument('--is_training', type=int, required=False, default=1, help='status')
parser.add_argument('--train_only', type=bool, required=False, default=False, help='perform training on full input dataset without validation and testing')
parser.add_argument('--model_id', type=str, required=False, default='test', help='model id')
parser.add_argument('--model', type=str, required=False, default='Autoformer',
                    help='model name, options: [Autoformer, Informer, Transformer]')

# data loader
parser.add_argument('--data', type=str, required=False, default='ETTm1', help='dataset type')
parser.add_argument('--root_path', type=str, default='./data/ETT/', help='root path of the data file')
parser.add_argument('--data_path', type=str, default='ETTh1.csv', help='data file')
parser.add_argument('--features', type=str, default='M',
                    help='forecasting task, options:[M, S, MS]; M:multivariate predict multivariate, S:univariate predict univariate, MS:multivariate predict univariate')
parser.add_argument('--target', type=str, default='OT', help='target feature in S or MS task')
parser.add_argument('--freq', type=str, default='h',
                    help='freq for time features encoding, options:[s:secondly, t:minutely, h:hourly, d:daily, b:business days, w:weekly, m:monthly], you can also use more detailed freq like 15min or 3h')
parser.add_argument('--checkpoints', type=str, default='./checkpoints/', help='location of model checkpoints')

# forecasting task
parser.add_argument('--seq_len', type=int, default=24, help='input sequence length')
parser.add_argument('--label_len', type=int, default=0, help='start token length')
parser.add_argument('--pred_len', type=int, default=24, help='prediction sequence length')


# DLinear
parser.add_argument('--individual', action='store_true', default=False, help='DLinear: a linear layer for each variate(channel) individually')
# Formers 
#Michi: bisher nur default=3 zum laufen gebracht
parser.add_argument('--embed_type', type=int, default=3, help='0: default 1: value embedding + temporal embedding + positional embedding 2: value embedding + temporal embedding 3: value embedding + positional embedding 4: value embedding')
parser.add_argument('--enc_in', type=int, default=V*2, help='encoder input size') # DLinear with --individual, use this hyperparameter as the number of channels
parser.add_argument('--dec_in', type=int, default=V, help='decoder input size')
parser.add_argument('--c_out', type=int, default=V, help='output size')
parser.add_argument('--d_model', type=int, default=d_model, help='dimension of model')
parser.add_argument('--n_heads', type=int, default=8, help='num of heads')
parser.add_argument('--e_layers', type=int, default=e_layers, help='num of encoder layers')
parser.add_argument('--d_layers', type=int, default=d_layers, help='num of decoder layers')
parser.add_argument('--d_ff', type=int, default=d_ff, help='dimension of fcn')
parser.add_argument('--moving_avg', type=int, default=25, help='window size of moving average')
parser.add_argument('--factor', type=int, default=1, help='attn factor')
parser.add_argument('--distil', action='store_false',
                    help='whether to use distilling in encoder, using this argument means not using distilling',
                    default=True)
parser.add_argument('--dropout', type=float, default=0.05, help='dropout')
parser.add_argument('--embed', type=str, default='timeF',
                    help='time features encoding, options:[timeF, fixed, learned]')
parser.add_argument('--activation', type=str, default='gelu', help='activation')
parser.add_argument('--output_attention', action='store_true', help='whether to output attention in ecoder')
parser.add_argument('--do_predict', action='store_true', help='whether to predict unseen future data')

# optimization
parser.add_argument('--num_workers', type=int, default=10, help='data loader num workers')
parser.add_argument('--itr', type=int, default=2, help='experiments times')
parser.add_argument('--train_epochs', type=int, default=10, help='train epochs')
parser.add_argument('--batch_size', type=int, default=32, help='batch size of train input data')
parser.add_argument('--patience', type=int, default=3, help='early stopping patience')
parser.add_argument('--learning_rate', type=float, default=0.0001, help='optimizer learning rate')
parser.add_argument('--des', type=str, default='test', help='exp description')
parser.add_argument('--loss', type=str, default='mse', help='loss function')
parser.add_argument('--lradj', type=str, default='type1', help='adjust learning rate')
parser.add_argument('--use_amp', action='store_true', help='use automatic mixed precision training', default=False)

# GPU
parser.add_argument('--use_gpu', type=bool, default=True, help='use gpu')
parser.add_argument('--gpu', type=int, default=0, help='gpu')
parser.add_argument('--use_multi_gpu', action='store_true', help='use multiple gpus', default=False)
parser.add_argument('--devices', type=str, default='0,1,2,3', help='device ids of multile gpus')
parser.add_argument('--test_flop', action='store_true', default=False, help='See utils/tools for usage')

args = parser.parse_args(args=[])
import importlib

loading of joblib completed


# 7 Sofa Analysis DMS Informer

# 10 Sofa Analysis IMS Informer Backprop

In [8]:
import os
import models.InformerAutoregressiveFullRegression as autoformer
importlib.reload(autoformer)
batch_size, lr, patience = 32, 0.0005, 6  # batch_size increased, patience 10 --> 6
d, N, he, dropout = 50, 2, 4, 0.2
model = autoformer.Model(args).cuda()
# Load pretrained weights here.
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
scheduler = ReduceLROnPlateau(optimizer, 'max', patience=3)
import math
for model_path in os.listdir("eicu_regression_results_high_lr/"):
    #if not (model_path.endswith("1.model") or model_path.endswith("2.model")): continue
    #if not ("k80" in model_path or "k160" in model_path) or not(model_path.endswith("1.model")): continue
    if not (model_path.endswith(".model")): continue
    print(model_path)
    model.load_state_dict(torch.load("eicu_regression_results_high_lr/"+model_path))
    model.eval()
    y_true, y_pred = [], []
    #print(test_input)
    pbar = tqdm(range(0, len(test_input), batch_size))
    loss_list = []
    test_loss = 0
    test_loss_part1 = 0
    test_loss_part2 = 0
    sofa_losses = []
    test_loss_sofa_variables = 0
    accuracy_list = []
    TP, TN, FP, FN = 0, 0, 0, 0
    test_loss_sum_square_total = 0
    test_loss_part1_sum_square_total = 0
    test_loss_part2_sum_square_total = 0
    test_loss_sofa_sum_square_total = 0
    for start in pbar:
        #print(start)
        matrix = torch.tensor(test_input[start:start+batch_size], dtype=torch.float32).cuda()
        input_matrix = matrix[:, :24]

        #torch.Size([32, 24, 129])
        input_mark = torch.arange(0, input_matrix.size(1)).unsqueeze(0).repeat(input_matrix.size(0), 1).cuda()
        #torch.Size([32, 24])
        input_mask = input_matrix[:, :24, V:]
        output_matrix = matrix[:, 24:, :V]
        #torch.Size([32, 24, 129])
        output_mark = torch.arange(0, output_matrix.size(1)).unsqueeze(0).repeat(input_matrix.size(0), 1).cuda()
        #torch.Size([32, 24])
        output_mask = matrix[:, 24:, V:]
        #torch.Size([32, 24, 129])
        output_mark = torch.arange(0, output_matrix.size(1)).unsqueeze(0).repeat(input_matrix.size(0), 1).cuda()
        dec_inp = torch.zeros_like(output_matrix[:, -args.pred_len:, :]).float()
        with torch.no_grad():
            output = model(input_matrix, input_mark, dec_inp, output_mark, trainn=False)
        for outpu, real, mask, input_mat in zip(output, output_matrix, output_mask, input_matrix):
            #print(outpu)
            a = outpu.item()
            second_day = sum(get_sofa(real, var_to_ind))
            #print(second_day)
            first_day = sum(get_sofa(input_mat, var_to_ind)) #SOFA Source (First Day)
            c, d = torch.tensor(a, dtype=torch.float32), torch.tensor(second_day, dtype=torch.float32)
            sofa_losses.append(((c-d)**2).sum(axis=-1).mean().item())

    sofa_loss = sum(sofa_losses)/len(sofa_losses)
    sofa_var = math.sqrt(sum(x**2 for x in sofa_losses)/len(sofa_losses) - sofa_loss**2)/math.sqrt(len(sofa_losses))
    model_name = model_path + "&"
    sofa_var = torch.tensor(sofa_var)
    SOFA_string = "{:.3f}".format(sofa_loss)+ '$_{['+"{:.3f}".format(sofa_loss-1.96*sofa_var.item())+','+"{:.3f}".format(sofa_loss+1.96*sofa_var.item())+ ']}$&'
    print(model_name, SOFA_string, sep='',file=open('regression_eicu_partial2.txt','a'))

cutmix-s43-SILVER_ONLY_True_n_246080000_2_PRETRAINED_MODEL_NONE.model


100%|██████████| 94/94 [00:22<00:00,  4.26it/s]


geca-cut_sadlo-k160_s43_w1_f4-SILVER_ONLY_True_n_246080000_2_PRETRAINED_MODEL_NONE.model


100%|██████████| 94/94 [00:21<00:00,  4.28it/s]


geca-cut_sadlo-k160_s42_w1_f4-SILVER_ONLY_True_n_246080000_1_PRETRAINED_MODEL_NONE.model


...

100%|██████████| 94/94 [00:21<00:00,  4.27it/s]


cutmix-s41-SILVER_ONLY_True_n_246080000_1_PRETRAINED_MODEL_NONE.model


100%|██████████| 94/94 [00:21<00:00,  4.28it/s]


cutmix-s42-SILVER_ONLY_True_n_246080000_1_PRETRAINED_MODEL_NONE.model


100%|██████████| 94/94 [00:21<00:00,  4.28it/s]


In [7]:
import os
import models.InformerAutoregressiveFullRegression as autoformer
importlib.reload(autoformer)
batch_size, lr, patience = 32, 0.0005, 6  # batch_size increased, patience 10 --> 6
d, N, he, dropout = 50, 2, 4, 0.2
model = autoformer.Model(args).cuda()
# Load pretrained weights here.
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
scheduler = ReduceLROnPlateau(optimizer, 'max', patience=3)
for model_path in os.listdir("eicu_regression_results_partial_data/"):
    #if not (model_path.endswith("1.model") or model_path.endswith("2.model")): continue
    #if not ("k80" in model_path or "k160" in model_path) or not(model_path.endswith("1.model")): continue
    if not (model_path.endswith(".model")): continue
    print(model_path)
    model.load_state_dict(torch.load("eicu_regression_results_partial_data/"+model_path))
    model.eval()
    y_true, y_pred = [], []
    #print(test_input)
    pbar = tqdm(range(0, len(test_input), batch_size))
    loss_list = []
    test_loss = 0
    test_loss_part1 = 0
    test_loss_part2 = 0
    sofa_losses = []
    test_loss_sofa_variables = 0
    accuracy_list = []
    TP, TN, FP, FN = 0, 0, 0, 0
    test_loss_sum_square_total = 0
    test_loss_part1_sum_square_total = 0
    test_loss_part2_sum_square_total = 0
    test_loss_sofa_sum_square_total = 0
    for start in pbar:
        #print(start)
        matrix = torch.tensor(test_input[start:start+batch_size], dtype=torch.float32).cuda()
        input_matrix = matrix[:, :24]

        #torch.Size([32, 24, 129])
        input_mark = torch.arange(0, input_matrix.size(1)).unsqueeze(0).repeat(input_matrix.size(0), 1).cuda()
        #torch.Size([32, 24])
        input_mask = input_matrix[:, :24, V:]
        output_matrix = matrix[:, 24:, :V]
        #torch.Size([32, 24, 129])
        output_mark = torch.arange(0, output_matrix.size(1)).unsqueeze(0).repeat(input_matrix.size(0), 1).cuda()
        #torch.Size([32, 24])
        output_mask = matrix[:, 24:, V:]
        #torch.Size([32, 24, 129])
        output_mark = torch.arange(0, output_matrix.size(1)).unsqueeze(0).repeat(input_matrix.size(0), 1).cuda()
        dec_inp = torch.zeros_like(output_matrix[:, -args.pred_len:, :]).float()
        with torch.no_grad():
            output = model(input_matrix, input_mark, dec_inp, output_mark, trainn=False)
        for outpu, real, mask, input_mat in zip(output, output_matrix, output_mask, input_matrix):
            #print(outpu)
            a = outpu.item()
            second_day = sum(get_sofa(real, var_to_ind))
            #print(second_day)
            first_day = sum(get_sofa(input_mat, var_to_ind)) #SOFA Source (First Day)
            c, d = torch.tensor(a, dtype=torch.float32), torch.tensor(second_day, dtype=torch.float32)
            sofa_losses.append(((c-d)**2).sum(axis=-1).mean().item())

    sofa_loss = sum(sofa_losses)/len(sofa_losses)
    sofa_var = math.sqrt(sum(x**2 for x in sofa_losses)/len(sofa_losses) - sofa_loss**2)/math.sqrt(len(sofa_losses))
    model_name = model_path + "&"
    sofa_var = torch.tensor(sofa_var)
    SOFA_string = "{:.3f}".format(sofa_loss)+ '$_{['+"{:.3f}".format(sofa_loss-1.96*sofa_var.item())+','+"{:.3f}".format(sofa_loss+1.96*sofa_var.item())+ ']}$&'

    print(model_name, SOFA_string, sep='',file=open('regression_eicu_partial2.txt','a'))

geca-cut_sadlo-k160_s42_w1_f4-SILVER_ONLY_True_n_246080000_2_PRETRAINED_MODEL_NONE0.05_other_seed.model


100%|██████████| 94/94 [00:21<00:00,  4.29it/s]


geca-cut_sadlo-k160_s42_w1_f4-SILVER_ONLY_True_n_246080000_2_PRETRAINED_MODEL_NONE0.25.model


100%|██████████| 94/94 [00:21<00:00,  4.29it/s]


geca-cut_sadlo-k160_s42_w1_f4-SILVER_ONLY_True_n_246080000_4_PRETRAINED_MODEL_NONE0.05.model


...

100%|██████████| 94/94 [00:21<00:00,  4.29it/s]


geca-cut_sadlo-k160_s42_w1_f4-SILVER_ONLY_True_n_246080000_6_PRETRAINED_MODEL_NONE0.05.model


100%|██████████| 94/94 [00:21<00:00,  4.29it/s]


geca-cut_sadlo-k160_s42_w1_f4-SILVER_ONLY_True_n_246080000_5_PRETRAINED_MODEL_NONE0.05_other_seed.model


100%|██████████| 94/94 [00:21<00:00,  4.29it/s]
